In [1]:
import numpy as np # no use until now
import random # used for random sampling of values where applicable
import math # no use untill now
import pandas as pd # to read the excel file 
import os # to carry os operations

In [2]:
def make_templates(file_name="templates_for_dialogue_self_play.xlsx",sheet_name="MAKE_TRANSACTION",previous_dictionary=None) :
    if previous_dictionary == None :
        template_dictionary = dict()
    else :
        template_dictionary = previous_dictionary
    
    df = pd.read_excel(file_name,sheet_name)
    for index,row in df.iterrows() :
        if not pd.isnull(row["LABEL"]) :
            if row["LABEL"] not in template_dictionary.keys() :
                template_sentences = list()
            else :
                template_sentences = template_dictionary[row["LABEL"]]
                
            if not pd.isnull(row["TEXT-EN (Marco)"]) :
                template_sentences.append(row["TEXT-EN (Marco)"])
            if not pd.isnull(row["TEXT-EN (Sourabh)"]) :
                template_sentences.append(row["TEXT-EN (Sourabh)"])
                
            template_dictionary[row["LABEL"]] = template_sentences
    
    return template_dictionary

In [3]:
account_balance_templates = make_templates(file_name="templates_for_dialogue_self_play.xlsx",
                                     sheet_name="ACCOUNT_BALANCE",
                                     previous_dictionary=None)

In [4]:
class Action(object) :
    def __init__(self,actor=None,action=None,slots=None,values=None,message=None,description=None,templates=None) :
        
        self.actor = actor # who performed the action
        self.action = action # what action was performed
        self.slots = slots # what slot was dealt with 
        self.values = values # what was the value with this slot
        self.message = message # Any particular message related to the action
        self.description = description # This contain the description of the action and is never intended to be shown or appear in the actual conversation
        
        dictionary_key = self.action
        
        if self.slots :
            if len(self.slots) > 0 :
                for slot in self.slots :
                    
                    if slot == "intent" :
                        
                        if self.actor == "User" :
                            
                            dictionary_key += "-" + slot + "_" + self.values["intent"]
                            
                        else :
                            
                            dictionary_key += "-" + slot
                    else :
                        
                        dictionary_key += "-" + slot
        
        # if the dictionary_key exists in the template dictionary then get a template other wise set template = the action message
        if dictionary_key in templates.keys() :
            self.template = random.sample(templates[dictionary_key],1)[0]
        else :
            self.template = self.message
        
        
    # standard actions to perform
    def get_actor(self) :
        return self.actor
    
    def get_action(self) :
        return self.action
    
    def get_slots(self) :
        return self.slots
    
    def get_values(self) :
        return self.values
    
    def get_message(self) :
        return self.message
    
    def get_description(self) :
        return self.description
    
    # when called , construct a dialog from the slots and give it to the user
    def get_dialog(self,with_actor=True) :

            
        # first split the template into a list of words
        words = self.template.strip().split()
        sentence = None
        
        # only go to this loop if the actor is a User or Bot , in case of API go to else statement which will just append action and message
        if self.actor == "User" or self.actor == "Bot" :
            if self.values :
                for slot,value in self.values.items() :
                    search_slot = "{" + slot + "}"
                    if search_slot in words :
                        slot_index = words.index(search_slot)
                        words.insert(slot_index,str(value))
                
                        words.pop(slot_index+1)
                    sentence = " ".join(words)
                
            # if it's a request action then get a template requesting the slots
            elif self.action == "request" :
                
                sentence = self.template
            
            # if it's a end_call then show the action and the message given
            elif self.action == "end_call" :
                
                sentence = self.action + " " + self.message
            
            else :
                
                sentence = self.message
        
        else:
            
            sentence = self.action + " " + self.message
        if with_actor :
            return self.actor + " : " + sentence
        else :
            return sentence

In [5]:
action = Action(actor="Bot",action="api_call",slots=["user_account","destination_name","amount"],
                values={"user_account" : "savings","destination_name" : "sourabh","amount" : "1000"},
                message="api_name:transaction, user_account:{}, destination_name:{}, amount:{}".format("savings","sourabh","1000"),
                description="API",templates=account_balance_templates)
print(action.get_dialog())

Bot : api_name:transaction, user_account:savings, destination_name:sourabh, amount:1000


In [7]:
class Account_user() :
    def __init__(self,templates=None) :
        
        # Below is the available pool of values from which we will create a Custom user for the transaction
        self.user_names = ["Sourabh","Serra","Simone","Marco","Vevake","Matteo","Tahir","Samuel"]
        self.user_accounts = ["Savings","Credit","Checkin"]
        self.user_balances = [400,1300,3000,8000]
        self.slots = ["user_account"]
        self.templates = templates
        
        self.priority_states = list()
        self.priority_actions = dict()
        
        # create the custom user
        self.user = dict()
        self.create_user_profile()
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    def create_user_profile(self) :
        
        # Every value is assigned randomly 
        
        # selectinng name of sender and reciever
        
        self.user["name"] = random.sample(self.user_names,1)[0]
                
        #selecting the usr_account to make the transaction from
        
        
        # select at random the number of account the user has.
        number_of_account = random.randint(1,len(self.user_accounts))
        
        self.user["user_accounts"] = random.sample(self.user_accounts,number_of_account)
        
        # select a list of accounts from the given sample
        
        self.user["user_account"] = random.sample(self.user_accounts,1)[0]
        
        self.user["balance"] = random.sample(self.user_balances,1)[0]
                        
        # setting up the intent
        self.user["intent"] = "account_balance"
    
    # Returns the respective value of the slot
    def get_value(self,slot_asked) :
        
        return self.user[slot_asked]
    
    # This function is called when the bot has made a request but no slots have been provided, hence we look at the description of the action to figure out what the request is
    def perform_random_action(self,bot_action) :
        
        user_action = None
        actual_actor = None
        actual_action = None
        accept_message = str()
        reject_message = str()
        values_to_give = dict()
        if bot_action.get_description() == "SELECT_ACCOUNT" :
               
            self.user["user_account"] = random.sample(self.user_accounts,1)[0]
            
            user_action = Action(actor="User",
                                action="inform",
                                slots=["user_account"],
                                values={"user_account" : self.user["user_account"]},
                                message="providing value for user_account",
                                templates=self.templates)
        
        else :
            
            if bot_action.get_description() == "API_CALL" :
                
                actual_actor = "API"
                actual_action = "api_response"
                accept_message = "api_result:success, balance:{}".format(self.user["balance"])
                reject_message = "api_result:failed"
                values_to_give = {"balance" : self.user["balance"]}
            
            elif bot_action.get_description() == "CHANGE_ACCOUNT" :
                
                actual_actor = "User"
                actual_action = "inform"
                accept_message = "accept"
                reject_message = "reject"
                
                new_account = random.sample(self.user_accounts,1)[0]
                
                while new_account == self.user["user_account"] :
                    new_account = random.sample(self.user_accounts,1)[0]
                
                self.user["user_account"] = new_account
                
            else :
                
                actual_actor = "User"
                actual_action = "inform"
                accept_message = "accept"
                reject_message = "reject"
            
            toss = random.randint(0,1)
            if toss == 1 :
                user_action = Action(actor=actual_actor,
                                     action=actual_action,
                                     slots=None,
                                     values=values_to_give,
                                     message=accept_message,
                                     templates=self.templates)
            else :
                
                user_action = Action(actor=actual_actor,
                                     action=actual_action,
                                     slots=None,
                                     values=values_to_give,
                                     message=reject_message,
                                     templates=self.templates)
        return user_action
    # This is the function that converses with the bot through 'Action' Objects
    def speak(self,bot_action) :
        user_action = None
        if bot_action.get_action() == "api_call" :
            
            user_action = self.api_response(bot_action)            

        elif bot_action.get_action() == "request" :
            
            if bot_action.get_slots() != None :
                
                if bot_action.get_slots()[0] != "intent" :
                    
                    user_value = self.get_value(bot_action.get_slots()[0])
                    user_action = Action(actor="User",
                                         action="inform",
                                         slots=bot_action.get_slots(),
                                         values={bot_action.get_slots()[0] : user_value},
                                         message="Providing value for {}".format(bot_action.get_slots()[0]),
                                         templates=self.templates)
                
                else :
                    
                    number_of_slots = random.randint(0,len(self.slots))
                    slots_to_inform = random.sample(self.slots,number_of_slots)
                    all_slots = ["intent"] + seslots_to_inform
                    values_to_inform = dict()
                    
                    for slot in all_slots :
                        values_to_inform[slot] = self.user[slot]
                    
                    values_to_inform["name"] = self.user["name"]
                        
                    user_action = Action(actor="User",
                                       action="inform",
                                       slots=all_slots,
                                       values=values_to_inform,
                                       message="Providing value for intent",
                                       templates=self.templates)
            else:
                
                user_action = self.perform_random_action(bot_action)
        
        
        else :
            
            user_action = Action(actor="User",
                                 action=None,
                                 slots=None,
                                 values=None,
                                 message="<SILENCE>",
                                 templates=self.templates)
        
        return user_action
    
    # when the bot takes the role of API then, the User should assume the role of API_RESP (i.e API_RESPONSE)
    def api_response(self,bot_action) :
    
        user_action = None
        
        # if the API action asks for a account check
        if bot_action.get_description() == "REQUEST_ACCOUNTS" :
            
            slot_message = ",".join(self.user["user_accounts"])
            bot_message = "list_of_accounts:{}".format(slot_message)
            user_action = Action(actor="API",
                                action="api_response",
                                slots = self.user["user_accounts"],
                                values=None,
                                message=bot_message,
                                description="LIST_OF_SLOTS",
                                templates=self.templates)
            
        elif bot_action.get_description() == "API_INITIAL_SLOT_CHECK" :
            flag = False
            error_message = list()
            
            if "user_account" in bot_action.get_slots() and self.user["user_account"] not in self.user["user_accounts"] :
                
                self.priority_states.append("check_account")
                self.priority_actions["check_account"] = Action(actor="Bot",
                                                                action="api_call",
                                                                slots=["user_account"],
                                                                values=None,
                                                                message="api_call:check_account_api user_account:{}".format(self.user["user_account"]),
                                                                description="API_ACCOUNT_CHECK",
                                                                templates=self.templates)
            if self.priority_states :
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=self.priority_states,
                                     values=self.priority_actions,
                                     message="api_result:failed, message:'user_account is incorrect'",
                                     templates=self.templates)
            else :
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=bot_action.get_slots(),
                                     values=None,
                                     message="api_result:success",
                                     templates=self.templates)
                
        elif bot_action.get_description() == "API_ACCOUNT_CHECK" :
            
            print("checking account")
            if self.user["user_account"] in self.user["user_accounts"] :
                
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=["account"],
                                     values=self.user,
                                     message="api_result:success",
                                     templates=self.templates)
            else :
                
                slot_message = ','.join(self.user["user_accounts"])
                api_message = "api_result:failed, message:'available list of accounts : {}'".format(slot_message)
                user_action = Action(actor="API",
                                     action="api_response",
                                     slots=self.user["user_accounts"],
                                     values=None,
                                     message=api_message,
                                     templates=self.templates)
            
        else :
            user_action = self.perform_random_action(bot_action)
        
        
        return user_action            

In [6]:
class Account_bot(object) :
    
    def __init__(self,templates=None) :
        
        self.last_slot = None
        self.list_of_slots = ["user_account","destination_name","amount"]
        self.slots_to_ask = ["user_account"]
        self.user_values = dict()
        self.states = {"initial" : self.initial_state ,
                       "check_initial" : self.check_initial_state,
                       "list_accounts" : self.list_accounts_state,
                       "user_account" : self.select_account_state,
                       "check_account" : self.check_account_state,
                       "change_account" : self.change_account_state,
                       "api_call" : self.api_call_state,
                       "end_call" : self.end_call_state}
        
        self.priority_states = list()
        self.priority_actions = dict()
        self.templates = templates
        
        self.current_state = self.initial_state
    
    def sort_my_slots(self,slots_given) :
        slots_sorted = list()
        
        if "user_account" in slots_given :
            slots_sorted.append("user_account")
            slots_given.remove("user_account")
        
        if "destination_name" in slots_given :
            slots_sorted.append("destination_name")
            slots_given.remove("destination_name")
        
        if "amount" in slots_given :
            slots_sorted.append("amount")
            slots_given.remove("amount")
        
        for slot in slots_given :
            slots_sorted.append(slot)
        
        return slots_sorted
    
    # store any new values given by the user
    def record_user_values(self,user_action) :
        
        if type(user_action.get_values()) == dict :
            
            for slot,values in user_action.get_values().items() :
                self.user_values[slot] = values
                
     # remove the slots given from the list of slots to ask           
    def remove_informed_slots(self,user_action) :
        
        if user_action.get_slots() :
            for slot in user_action.get_slots() :
                if slot in self.slots_to_ask :
                    self.slots_to_ask.remove(slot)
                
    def speak(self,user_action) :
        
        if user_action == None :
            
            print("user_action received is None")
        
        next_state , bot_action = self.current_state(user_action)
        print("next_state is {}".format(next_state))
        self.current_state = self.states[next_state]
        
        return bot_action
        
    # meet the initial state, here the user may provide one or more than one values
    # request intent if not given already
    def initial_state(self,user_action) :
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_slots() :
            if "intent" in user_action.get_slots() and len(user_action.get_slots()) > 1 :
                
                next_state = "check_initial"
                slots_given = user_action.get_slots()[1:]
                slot_message = "api_call:initial_slot_check,"
                for slot in slots_given :
                    slot_message += " {}:{},".format(slot,self.user_values[slot])
                
                bot_action = Action(actor="Bot",
                                    action="api_call",
                                    slots=user_action.get_slots(),
                                    values=None,
                                    message=slot_message[:-1],
                                    description="API_INITIAL_SLOT_CHECK",
                                    templates=self.templates)
                
            else :
                
                next_state = "list_accounts"
                bot_action = Action(actor="Bot",
                                   action="api_call",
                                   slots=["name"],
                                   values=None,
                                   message="api_call:request_account_api, accounts:{}".format(self.user_values["name"]),
                                   description="REQUEST_ACCOUNTS",
                                   templates=self.templates)
        else :
            
            next_state = "initial"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=["intent"],
                               values=None,
                               message="requesting the intent from the user",
                               templates=self.templates)
        return next_state , bot_action
    
    def check_initial_state(self,user_action) :
        
        if user_action.get_message() == "api_result:success" :
            if not self.slots_to_ask :
                next_state = "api_call"
                api_value = self.user_values["user_account"]
                api_message = "api_call:check_balance_api, user_account:{}".format(self.user_values["user_account"])
                bot_action = Action(actor="Bot",
                                   action="api_call",
                                   slots=None,
                                   values={"api_call" : api_value},
                                   message=api_message,
                                   description="API_CALL",
                                   templates=self.templates)
            else :
                next_state = self.slots_to_ask[0]
                
        else :
            self.priority_states = user_action.get_slots()
            self.priority_actions = user_action.get_values()
            
            next_state = self.priority_states[0]
            bot_action = self.priority_actions[next_state]
            
            self.priority_states.remove(next_state)
            
        return next_state , bot_action
    
    def list_accounts_state(self,user_action) :
        
        if user_action.get_description() == "LIST_OF_SLOTS" :
            next_state = "user_account"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "You have the following accounts : {} , which one do you wish ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="SELECT_ACCOUNT",
                                templates=self.templates)
        else : 
            next_state = "list_accounts"
            bot_action = Action(actor="Bot",
                               action="api_call",
                               slots=["name"],
                               values=None,
                               message="api_call:request_accounts_api, accounts:{}".format(self.user_values["name"]),
                               description="REQUEST_ACCOUNTS",
                               templates=self.templates)
            
        return next_state , bot_action
        
    def select_account_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if "user_account" in user_action.get_slots() :
            
            next_state = "check_account"
            bot_action = Action(actor="Bot",
                                action="api_call",
                                slots=None,
                                values=None,
                                message="api_call:check_account_api, user_account:{}".format(self.user_values["user_account"]),
                                description="API_ACCOUNT_CHECK",
                                templates=self.templates)
        else :
            next_state = "user_account"
            bot_action = Action(actor="Bot",
                               action="request",
                               slots=None,
                               values=None,
                               message="Please select an account",
                               description="SELECT_ACCOUNT",
                               templates=self.templates)
        
        return next_state , bot_action
    
    def check_account_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_message().startswith("api_result:success") :
            
            next_state = "api_call"
            api_value = self.user_values["user_account"]
            api_message = "api_call:check_balance_api, user_account:{}".format(self.user_values["user_account"])
            bot_action = Action(actor="Bot",
                               action="api_call",
                               slots=None,
                               values={"api_call" : api_value},
                               message=api_message,
                               description="API_CALL",
                               templates=self.templates)
        else :
            next_state = "change_account"
            slot_message = ",".join(user_action.get_slots())
            bot_message = "It seems that you have not entered a valid account, you available accounts are {}, would you like change the source account ?".format(slot_message)
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=None,
                                values=None,
                                message=bot_message,
                                description="CHANGE_ACCOUNT",
                                templates=self.templates)
        return next_state , bot_action
    
    def change_account_state(self,user_action) :
        if user_action.get_message() == "accept" :
            next_state = "user_account"
            bot_action = Action(actor="Bot",
                                action="request",
                                slots=[next_state],
                                values=None,
                                message="Requesting user to provide new user account",
                                templates=self.templates)
        else :
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                                action="end_call",
                                slots=None,
                                values=None,
                                message="Denied to change account",
                                templates=self.templates)
        
        return next_state , bot_action
    
    def api_call_state(self,user_action) :
        
        self.record_user_values(user_action)
        self.remove_informed_slots(user_action)
        
        if user_action.get_message().startswith("api_result:success") :
            
            next_state = "end_call"
            bot_message = "your current balance for account:{} is {}".format(self.user_values["user_account"],self.user_values["balance"])
            bot_action = Action(actor="Bot",
                                action="end_call",
                               slots=None,
                               values=None,
                               message=bot_message,
                               templates=self.templates)
        else :
            
            next_state = "end_call"
            bot_action = Action(actor="Bot",
                               action="end_call",
                               slots=None,
                               values=None,
                               message="error in processing request !!",
                               templates=self.templates)
        
        return next_state , bot_action
    
    def end_call_state(self,user_action) :
        print("Reached end of transaction")
        return

In [8]:
def create_dialogs(User,Bot,number_of_dialogs,dialog_templates=None) :
    
    dialogs = list()
    
    for i in range(number_of_dialogs) :
        
        dialog = list()

        user = User(templates=dialog_templates)
        bot = Bot(templates=dialog_templates)

        user_action = Action(actor="User",
                             action=None,
                             slots=None,
                             values=None,
                             message="<SILENCE>",
                             templates=dialog_templates)
        

        bot_action = Action(actor="Bot",
                            action="request",
                            slots=["intent"],
                            values=None,
                            message="Gettinng intent",
                            templates=dialog_templates)
        
        
        
        dialog.append(user_action)
        dialog.append(bot_action)
        
        latest_action = None
        
        while latest_action != "end_call" :
            user_action = user.speak(bot_action)
            #print("user_action {}, user_message {} user_description {}".format(user_action.get_action(),user_action.get_message(),user_action.get_description()))
            bot_action = bot.speak(user_action)
            #print("bot_action {}, bot_message {} bot_description {}".format(bot_action.get_action(),bot_action.get_message(),bot_action.get_description()))
            latest_action = bot_action.get_action() 
            #print("latest action is {}".format(latest_action))
            dialog.append(user_action)
            dialog.append(bot_action)
            #print("User:{} Bot:{}".format(user_action.get_message(),bot_action.get_message()))
        
        dialogs.append(dialog)
    
    return dialogs

In [9]:
account_dialogs = create_dialogs(User=Account_user,Bot=Account_bot,number_of_dialogs=100,dialog_templates=account_balance_templates)

next_state is list_accounts
next_state is user_account
next_state is check_account
checking account
next_state is api_call
next_state is end_call
next_state is list_accounts
next_state is user_account
next_state is check_account
checking account
next_state is change_account
next_state is user_account
next_state is check_account
checking account
next_state is api_call
next_state is end_call
next_state is list_accounts
next_state is user_account
next_state is check_account
checking account
next_state is api_call
next_state is end_call
next_state is check_initial
next_state is api_call
next_state is end_call
next_state is check_initial
next_state is api_call
next_state is end_call
next_state is list_accounts
next_state is user_account
next_state is check_account
checking account
next_state is api_call
next_state is end_call
next_state is list_accounts
next_state is user_account
next_state is check_account
checking account
next_state is change_account
next_state is user_account
next_state 

In [10]:
def create_raw_data(file_directory="../data/",file_name="data.txt",dialogs=None) :
    
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
    
    file_handle = open(os.path.join(file_directory,file_name),"w")
    
    for dialog in dialogs :
        for action in dialog :
            file_handle.write(action.get_dialog())
            file_handle.write("\n")
        file_handle.write("\n")
    file_handle.close()

In [12]:
create_raw_data(file_directory="../data/account_balance_data/",file_name="raw_data.txt",dialogs=account_dialogs)

In [11]:
def create_training_data(file_directory="../data/",file_name="data.txt",dialogs=None) :
    
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
        
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for dialog in dialogs :
        count = 1
        for i in range(0,len(dialog),2) :
            file_handle.write("{} {}\t{}\n".format(str(count),dialog[i].get_dialog(with_actor=False),dialog[i+1].get_dialog(with_actor=False)))
            count += 1
        file_handle.write("\n")
    file_handle.close()

In [14]:
create_training_data(file_directory="../data/account_balance_data/",file_name="train_data.txt",dialogs=account_dialogs)
create_training_data(file_directory="../data/account_balance_data/",file_name="val_data.txt",dialogs=account_dialogs)
create_training_data(file_directory="../data/account_balance_data/",file_name="test_data.txt",dialogs=account_dialogs)

In [15]:
def create_candidates(file_directory="../data/",file_name="data.txt",dialogs=None) :
    candidate_set = set()
    if not os.path.exists(file_directory) :
        os.makedirs(file_directory)
        
    for dialog in dialogs :
        for action in dialog :
            if action.get_actor() == "Bot" or action.get_actor() == "API" :
                candidate_set.add(action.get_dialog(with_actor=False))
    file_handle = open(os.path.join(file_directory,file_name),"w")
    for candidate in candidate_set :
        file_handle.write("1 {}\n".format(candidate))
    file_handle.close()

In [16]:
create_candidates(file_directory="../data/account_balance_data/",file_name="candidates.txt",dialogs=account_dialogs)